# Gaussian impurity with self consistency

In [1]:
import kwant
import numpy as np
from numpy import linalg as la
from amsc import (
    bulk_amsc_system,
    generate_intial_Delta,
    setup_gaussian_impurities,
)
from tqdm.notebook import tqdm
from pauli import *
from qm_tools import sparse_diag
from scipy.interpolate import RegularGridInterpolator

In [2]:
# eigsh = sla.eigsh
eigsh = sparse_diag

In [3]:
periodic_bc = False

Nx = 61
Ny = 61

x_ax = np.linspace(-(Nx // 2), (Nx // 2), Nx)
y_ax = np.linspace(-(Ny // 2), (Ny // 2), Ny)
x, y = np.meshgrid(x_ax, y_ax)

t = 5.0
t_so = 0.0 * t
t_am = 0.25 * t

mu = 2.5
hz0 = 1e-6
hx0 = 0
hy0 = 0

## Initial value for Delta (will be used to determine g)
Delta_init = 1

In [4]:
# Impurituy positions
impurity_positions = [(0, 0)]
impurity_sizes = [2.2]
impurity_eccentricities = [0.0]
impurity_orientation = [0.0]

hx_imp = [0.0]
hy_imp = [0.0]
hz_imp = [0.0]

In [5]:
vortex_positions = []
windings = []
l_core = 0

Delta_0, theta_0 = generate_intial_Delta(
    x=x,
    y=y,
    Delta_init=Delta_init,
    vortex_positions=vortex_positions,
    windings=windings,
    l_core=l_core,
)

In [6]:
N = 1024
g = 10.2

In [19]:
def solve_for_V(
    Vs,
    g,
    Delta_0,
    theta_0,
    tol,
    maxiter,
    mixing,
    T,
):
    """
    Solve the self-consistent problem for an impurity in an altermagnetic superconductor
    for a given value of Vs.
    """

    # Define the potential field
    V, hx, hy, hz = setup_gaussian_impurities(
        x=x,
        y=y,
        mu=mu,
        hx0=hx0,
        hy0=hy0,
        hz0=hz0,
        impurity_sizes=impurity_sizes,
        impurity_positions=impurity_positions,
        impurity_eccentricities=impurity_eccentricities,
        impurity_orientations=impurity_orientation,
        V_imp=[Vs],
        hx_imp=hx_imp,
        hy_imp=hy_imp,
        hz_imp=hz_imp,
    )

    syst, lat = bulk_amsc_system(
        Nx=Nx,
        Ny=Ny,
        t=t,
        t_so=t_so,
        t_am=t_am,
        V=V,
        Delta=Delta_0,
        theta=theta_0,
        hx=hx,
        hy=hy,
        hz=hz,
        periodic_bc=periodic_bc,
    )

    fsyst = syst.finalized()

    # Singlet correlations operator
    txs0_op = kwant.operator.Density(fsyst, onsite=txs0, sum=False)
    tys0_op = kwant.operator.Density(fsyst, onsite=tys0, sum=False)

    Delta = Delta_0
    theta = theta_0

    Delta_n = Delta(x, y).flatten()
    theta_n = theta(x, y).flatten()

    for iter_number in range(maxiter):
        H = fsyst.hamiltonian_submatrix(
            params=dict(Delta=Delta, theta=theta), sparse=True
        )

        # Diagonalize the sytem
        ws, vs = eigsh(H, k=N, sigma=0)

        sort_idxs = np.argsort(ws)
        vs = vs[:, sort_idxs]
        ws = ws[sort_idxs]

        # Calculate correlation functions
        txs0_ev = np.zeros((N, Nx * Ny))
        tys0_ev = np.zeros((N, Nx * Ny))

        for i in range(N):
            txs0_ev[i] = txs0_op(vs[:, i])
            tys0_ev[i] = tys0_op(vs[:, i])

        Fx = np.einsum("ni, n -> i", txs0_ev, np.tanh(ws / (2 * T)))
        Fy = np.einsum("ni, n -> i", tys0_ev, np.tanh(ws / (2 * T)))

        # Calculate the order parameter
        Delta_n_new = g * np.sqrt(Fx**2 + Fy**2)
        theta_n_new = np.arctan2(Fy, Fx)

        Delta_n = (1 - mixing) * Delta_n + mixing * Delta_n_new
        theta_n = (1 - mixing) * theta_n + mixing * theta_n_new

        diff = np.mean((Delta_n.reshape(Nx, Ny) - Delta(x, y)) ** 2)
        print(f"Iteration {iter_number:2d}, the average error is: {diff:5f}")

        # Create the new interpolation functions
        Delta_interp = RegularGridInterpolator((y_ax, x_ax), Delta_n.reshape(Ny, Nx))
        theta_interp = RegularGridInterpolator((y_ax, x_ax), theta_n.reshape(Ny, Nx))

        # Update the order parameter
        Delta = lambda x, y: Delta_interp((y, x))
        theta = lambda x, y: theta_interp((y, x))

        if diff < tol:
            break

    return Delta_n, theta_n, ws, vs

In [20]:
Vss = np.arange(8, 25.5, 0.5)
NVs = len(Vss)

In [21]:
for i, Vs in tqdm(enumerate(Vss), total=len(Vss)):
    print(f"V = {-Vs}")

    Delta_n, theta_n, ws, vs = solve_for_V(
        Vs=Vs,
        g=g,
        Delta_0=Delta_0,
        theta_0=theta_0,
        tol=1e-7,
        maxiter=10,
        mixing=0.75,
        T=1e-3,
    )

    file_path = f"./data2/data_V{int(Vss[i]*10):03d}.npz"
    np.savez(file_path, Delta_n=Delta_n, theta_n=theta_n, ws=ws, vs=vs)

  0%|          | 0/35 [00:00<?, ?it/s]

V = -8.0
Iteration  0, the average error is: 0.280509
Iteration  1, the average error is: 0.017532
Iteration  2, the average error is: 0.001096
Iteration  3, the average error is: 0.000068
Iteration  4, the average error is: 0.000004
Iteration  5, the average error is: 0.000000
Iteration  6, the average error is: 0.000000
V = -8.5
Iteration  0, the average error is: 0.280606


KeyboardInterrupt: 